In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, datasets, layers
from keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np
import random
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import talib as tb
import time
import os
# import pywhatkit
import matplotlib.pyplot as plt
#from warnings import simplefilt

2023-12-25 23:15:55.327712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 23:15:55.566815: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 23:15:55.568146: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 23:15:56.884965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Training data is the data of the previous day<br>
i.e. we are considering trading only at the opening and closing of the market

In [ ]:
n50_symbols=np.array(['NIFTY', 'HINDALCO', 'MARUTI', 'NESTLEIND', 'ONGC', 'TATAMOTORS', 'ITC', 'SUNPHARMA', 'BHARTIARTL', 'CIPLA', 'DIVISLAB', 'JSWSTEEL', 'HINDUNILVR', 'TITAN', 'NTPC', 'TATASTEEL', 'HEROMOTOCO', 'HDFCLIFE', 'ULTRACEMCO', 'SBILIFE', 'TCS', 'BAJFINANCE', 'BRITANNIA', 'AXISBANK', 'TECHM', 'COALINDIA', 'LT', 'DRREDDY', 'M_M', 'APOLLOHOSP', 'ADANIPORTS', 'BAJAJFINSV', 'GRASIM', 'TATACONSUM', 'ICICIBANK', 'HDFCBANK', 'EICHERMOT', 'BPCL', 'ADANIENT', 'UPL', 'POWERGRID', 'HDFCBANK', 'INFY', 'WIPRO', 'RELIANCE', 'INDUSINDBK', 'ASIANPAINT', 'HCLTECH', 'KOTAKBANK', 'BAJAJ_AUTO', 'SBIN'])

folder_name = 'Daily_historical_data'

X_train=[]
y_train=[]
y_train_pos=[]
gain=[]


for i in range(51):
    file_name = f'{n50_symbols[i]}.csv'
    file_path = os.path.join(folder_name, file_name)
    df=pd.read_csv(file_path)
    df['datetime']=pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    # df['month'] = df['datetime'].dt.month
    # df['day_of_week'] = df['datetime'].dt.dayofweek
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute

    start_ind=[]
    for j in range(len(df.index)):
        if df['hour'][j]==9 and df['minute'][j]==15: # Getting the indices where the trading day starts
            start_ind.append(j)
    end_ind=[] # end of the day indices
    for j in range(1,len(start_ind)): #starting from 1 because we dont want an end without the start of a day which is the case for start_of_day[0]-1
        end_ind.append(start_ind[j]-1)

    # print(start_ind)
    # print(end_ind)    

    if len(start_ind)>len(end_ind):
        start_ind=start_ind[:-1]

    # print(start_ind)
    # print(end_ind)
    j=0
    while j<len(start_ind):
        if end_ind[j]-start_ind[j]!=74:
            end_ind.remove(end_ind[j])
            start_ind.remove(start_ind[j])
            continue
        #print(df.iloc[start_ind[j]+2:end_ind[j]-2])
        df_new=df.iloc[start_ind[j]+2:end_ind[j]-2].drop(['symbol','datetime','year','hour','minute'],axis=1)
        X_train.append(df_new.values.flatten())
        j+=1
    print(i)

    for j in end_ind:
        gain_inst=(df['close'][j+2]-df['close'][j-2])*100/df['close'][j-2] # gain at the particular instant
        gain.append(gain_inst)  # i+2 indicates the next day at 9:20(as we need to let the price settle) and i-2 at time 3:15 as we need to trade before the day ends and we cant trade ecaxtly at the end of the day
        if gain_inst>=0.1:
            y_train_pos.append(1)
        elif gain_inst<-0.1:
            y_train_pos.append(-1)
        else:
            y_train_pos.append(0)
            
y_train=gain